In [ ]:
!pip install -q keras==2.8.0
!pip install -q tensorflow==2.8.0

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from collections import deque, defaultdict
from imblearn.over_sampling import SMOTE
from warnings import filterwarnings
from xgboost import XGBClassifier
from scipy.special import softmax
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from typing import Callable
import tensorflow as tf
from sklearn import svm
from typing import List
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import logging
import pickle
import random
import keras
import copy
import json
import sys
import os
import gc

logging.getLogger("tensorflow").setLevel(logging.ERROR)
sns.set(rc = {'figure.figsize':(22,12)}, style="whitegrid")
filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'

In [ ]:
sys.path.insert(0,code_path)
from genetic_programming import SymbolicRegressor
from binirizer import CustomLabelBinirizer
from ensemble import Ensemble, Classifier
from oselm import OSELMClassifier,set_use_know
from DynamicFeatureSelection import dynamic_feature_selection
from SharedFunctions import prepare_data,train_and_test,feature_evolving,save_pickle,load_pickle,save_object,load_object,generate_new_samples

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# epsilon=0.97
# epsilon_decay=0.040
# # min_epsilon = 0.01
# episod = 200
# e = []
# for i in range(episod):
#     epsilon -= epsilon *  epsilon_decay
#     e.append(epsilon)
# print(epsilon)
# x = [ i for i in range(episod)]
# plt.plot(range(episod), e,label=f'epsilon decay policy')
# plt.xlabel('steps')
# plt.ylabel('epsilon')
# plt.savefig('0-25.svg', format='svg')
# plt.show()

In [ ]:
def epsilon_greedy(expected_reward, epsilon=0.97) -> int:
    """
    expected_reward: list of expected rewards for each possible action
    epsilon: .
    """
    if np.random.rand() <= epsilon:
        return np.random.choice(list(range(len(expected_reward))))
    else:
        return np.argmax(expected_reward)
PolicyFunction  = Callable[[np.ndarray, float], int]

In [ ]:
LEARNING_RATE = 0.001
def create_model(input_dim):
    K.clear_session()
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(input_dim,)))
    model.add(keras.layers.Dense(32, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(16, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(2,))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
class Agents:
    def __init__(self, evaluation_network,number_of_featuer, buffer_size: int = 800):

        self.evaluation_network = evaluation_network
        self.target_network = copy.deepcopy(self.evaluation_network)
        self.buffer_size = buffer_size
        self.fitted = False
        self.number_of_featuer = number_of_featuer
        self.reply_buffer = deque(maxlen=self.buffer_size)
        self.contrbution = np.random.rand()

    def make_action(self, curr_state: np.ndarray, policy_function: PolicyFunction, epsilon) -> int:
        if self.fitted:
            q_values = self.evaluation_network.predict(curr_state.reshape(-1, self.number_of_featuer),verbose=0)
            action = policy_function(q_values, epsilon)
        else:
            action = policy_function([0, 1], 1)
        return action

    def update_target_network(self):
        self.target_network = copy.deepcopy(self.evaluation_network)
        return

In [ ]:
class AgentsSoftmax(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsSoftmax.agent_count
        AgentsSoftmax.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)
        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        change_frequency = 0
        for state, next_state, reward, next_reward in zip(X_train[:-1], X_train[1:], rewards[: -1], rewards[1:]):
            if np.abs(state[self.agent_id] - next_state[self.agent_id]) == 1:
                self.contrbution += np.abs(reward - next_reward)
                change_frequency += 1
        self.contrbution = 0 if change_frequency==0 else self.contrbution/change_frequency
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsRegression(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsRegression.agent_count
        AgentsRegression.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsAverage(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsAverage.agent_count
        AgentsAverage.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        WINDOW_SIZE = 4
        X_train_ = np.zeros((X_train.shape[0] // WINDOW_SIZE, X_train.shape[1]))
        y_train_ = []
        j = 0
        for i in range(0, batch_size, WINDOW_SIZE):
            window_of_states = X_train[i: i + WINDOW_SIZE].sum(axis=0) / WINDOW_SIZE
            window_of_rewards = sum(rewards[i: i + WINDOW_SIZE])
            r = window_of_rewards * window_of_states[self.agent_id]
            X_train_[j, :] = np.around(window_of_states)
            if window_of_states[self.agent_id] == 0:
                if window_of_rewards > 0.6:
                    r = window_of_rewards
                else:
                    r = window_of_rewards / WINDOW_SIZE
            y_train_.append(r)
            j += 1
        X_train = X_train_
        y_train = np.array(y_train_)
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsSingle(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsSingle.agent_count
        AgentsSingle.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)
        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
def replay_buffer_decentralize(self, batch_size, model):
        minibatch = random.sample(self.memory, batch_size)
        target = 0
        for exploitation, state, action, reward, next_state in minibatch:
            target = reward
            if next_state is not None:
                next_state = np.array(next_state).reshape([1, np.array(next_state).shape[0]])
                logit_value = model.predict(next_state, verbose=0)[0]
                target = reward + self.gamma * np.amax(logit_value)
            state = np.array(state).reshape([1, np.array(state).shape[0]])
            target_f = model.predict(state, verbose=0)
            target_f[0][action] = target
            model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.min_epsilon:
            self.epsilon -= self.epsilon * self.epsilon_decay
        return target

In [ ]:
def generate_oselm_models(number_of_hidden_neurons, apply_model_replacement=False):
    models= [OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             ]

    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

def generate_ml_models(number_of_hidden_neurons, apply_model_replacement=False):
    models = [
              KNeighborsClassifier(5),
              KNeighborsClassifier(5),
              LogisticRegression(),
              LogisticRegression(),
              GaussianNB(),
              GaussianNB(),
              GaussianNB(),
              ]
    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

In [ ]:
# test set percentage
TESTSIZE=0.2
def get_reward(X,Y ,subset_features,apply_model_replacement=True):

    if sum(subset_features)==0:return -10
    global TESTSIZE
    subset_features = np.where(np.array(subset_features) == 1)[0]
    if subset_features.shape[0] == 0:return -5

    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    X_train, X_test = X_train[:,subset_features], X_test[:,subset_features]

    OSELM_Classifier = OSELMClassifier(X.shape[1]*3 // 2, 'relu', binarizer=CustomLabelBinirizer(), random_state=42)

    OSELM_Classifier.fit(X_train, y_train)

    y_pred1 = OSELM_Classifier.predict(X_test)

    # acc1 = accuracy_score(y_test, y_pred1)
    reward = f1_score(y_test, y_pred1)

    return reward

In [ ]:
def reward_strategy(time_step: int, accuracy: float, accuracy_history: list, subset_features: list, error_rate: float,
                    beta: float = 0.99):

    if sum(subset_features) == len(subset_features):
        return -5,-1

    elif sum(subset_features) == 0:
        return -10,-1

    elif accuracy > max(accuracy_history):
        return accuracy,1

    elif accuracy < max(accuracy_history):
        return -accuracy,-1

    else:
        return -1 * (beta * error_rate + ((1 - beta) * (sum(subset_features) / len(subset_features)))),-1

In [ ]:
def save_object(obj, filename,path):
    """
    _ INPUT (obj) THE OBJECT WE NEED SAVW IT (filename) THE NAME OF OBJECT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    """
    _ INPUT THE NAME OF OBJECT WE NEED LOAD IT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [ ]:
def softmax_distrbution(agents):
    contrbutions = []
    for agent in agents:
        contrbutions.append(agent.contrbution)
    return softmax(contrbutions)

def random_forest_distrbution(X_train,Y_train,num_of_agents, num_of_samples=1000):
    X,y = [],[]
    for i in range(num_of_samples):
        features_space = np.random.choice([0, 1], size=(num_of_agents,)).tolist()
        accuracy = get_reward(X_train,Y_train, features_space)
        X.append(features_space)
        y.append(accuracy)
    X = np.array(X)
    y = np.array(y)
    rf = RandomForestRegressor(n_estimators=15)
    rf.fit(X, y)
    return rf.feature_importances_.tolist()

In [ ]:
def prepare_data(csv_filename, target_column_name='class'):
    df = pd.read_csv(csv_filename)
    df = df.iloc[:80000, :]
    column_names = df.columns.tolist()
    if target_column_name not in column_names:
        target_column_name = column_names[-1]
    unique_vlaues = sorted(df[target_column_name].unique().tolist())
    df[target_column_name] = df[target_column_name].apply(lambda x: 0 if x == unique_vlaues[0] else 1)
    df[target_column_name] = df[target_column_name].astype('int')
    num_of_columns = len(column_names)
    df.columns = list(range(num_of_columns))
    return df

In [ ]:
def feature_selection(algo_type, agents, X, Y, eposide=200):
    epsilon=0.97
    epsilon_decay=0.040
    NUMBER_OF_AGENTS = X.shape[1]
    best_features = []
    features_space = np.random.choice([0, 1], size=(NUMBER_OF_AGENTS,)).tolist()
    contrbutions = []
    if algo_type == 'random_forest':
        contrbutions = random_forest_distrbution(X,Y,NUMBER_OF_AGENTS)
    elif algo_type in ['single_agent', 'average']:
        contrbutions = [1] * NUMBER_OF_AGENTS
    for i in tqdm(range(eposide)):
        rewards = [0]
        next_feature_space = []
        if algo_type == 'softmax':
            contrbutions = softmax_distrbution(agents)
        for t in range(0, NUMBER_OF_AGENTS):
            action = agents[t].make_action(np.array(features_space.copy()), epsilon_greedy, epsilon)
            next_feature_space.append(action)
            if algo_type == 'single_agent':
                features_space[t] = action
        reward_as_accuracy = get_reward(X,Y, next_feature_space)
        reward_at_time_t,best = reward_strategy(t, reward_as_accuracy, rewards, next_feature_space, 1 - reward_as_accuracy)
        if best>0:
          best_features = next_feature_space
        rewards.append(reward_as_accuracy)
        total_reward = reward_at_time_t
        transition = []
        for t in range(0, NUMBER_OF_AGENTS):
            transition.clear()
            feature_space_copy = features_space.copy()
            transition.append(feature_space_copy)
            action = next_feature_space[t]
            transition.append(action)
            transition.append(next_feature_space)
            transition.append(total_reward * contrbutions[t])
            agents[t].reply_buffer.append(transition)
            if len(agents[t].reply_buffer) > 32 and i % 32 == 0:
                agents[t].update_evaluation_network()
        if i % 64 == 0:
            for agent in agents:
                if agent.fitted:
                    agent.update_target_network()
        epsilon -= epsilon *  epsilon_decay
        features_space = next_feature_space
    return best_features if len(best_features)>0 else next_feature_space

In [ ]:
def generate_new_samples(buffer, y_values, n=500, y_col='label'):
    if not y_col in buffer.columns.tolist():
      y_col = buffer.columns.tolist()[-1]
    if y_values.sum() == 0:
       return buffer[buffer[y_col] == 1].sample(n, random_state=41)[:, :-1].values, np.array([1] * n)
    else:
      return buffer[buffer[y_col] == 0].sample(n,random_state=41)[:, :-1].values, np.array([0] * n)

In [ ]:
def save_pickle(obj, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(obj, f)
def load_pickle(file_name):
  with open(file_name, 'rb') as f:
    d = pickle.load(f)
  return d

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

def model_evaluation(X_train, X_test, y_train, y_test, selected_features):
    subset_features = np.where(np.array(selected_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    X_train, X_test = X_train[:,subset_features], X_test[:,subset_features]
    model = OSELMClassifier(X_train.shape[1]*3 // 2, 'relu', binarizer=CustomLabelBinirizer(), random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f1,recall,precision,accuracy)
    return f1,recall,precision,accuracy,y_pred

In [ ]:
def dynamic_feature_selection(chunk_X, chunk_Y,X_test,y_test,save_path,algorithm_type=['softmax','average','single_agent','random_forest'],chunk_number=0):
  AgentsSoftmax.agent_count,AgentsAverage.agent_count,AgentsSingle.agent_count,AgentsRegression.agent_count = 0,0,0,0
  softmax_agents,average_agents,single_agent_agents,random_forest_agents,result,over_all=[],[],[],[],[],[]
  NUM_OF_FEATURES = chunk_X.shape[1]
  for i in range(NUM_OF_FEATURES):

    if 'random_forest' in algorithm_type:
      random_forest_agents.append(AgentsSoftmax(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'average' in algorithm_type:
      average_agents.append(AgentsAverage(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'single_agent' in algorithm_type:
      single_agent_agents.append(AgentsSingle(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'softmax' in algorithm_type:
      softmax_agents.append(AgentsRegression(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))


  softmax_results,average_results,single_agent_results,random_forest_results,voting_results = results_dic(save_path,chunk_number)

  if 'softmax' in algorithm_type:
    softmax_result = feature_selection('softmax', softmax_agents,chunk_X, chunk_Y)
    f1, recall ,precision ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, softmax_result)
    over_all.append(f1)
    save_object(softmax_result, "softmax_mask_"+str(chunk_number),save_path)
    softmax_results["f1"].append(f1)
    softmax_results["recall"].append(recall)
    softmax_results["precision"].append(precision)
    softmax_results["accuracy"].append(accuracy)
    softmax_results["y_predicte"].append(y_predicte)
    save_object(softmax_results, "softmax_results",save_path)
    gc.collect()


  if 'average' in algorithm_type:
    average_result = feature_selection('average', average_agents,chunk_X, chunk_Y)
    f1, recall ,precision ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, average_result)
    over_all.append(f1)
    save_object(average_result, "average_mask_"+str(chunk_number),save_path)
    average_results["f1"].append(f1)
    average_results["recall"].append(recall)
    average_results["precision"].append(precision)
    average_results["accuracy"].append(accuracy)
    average_results["y_predicte"].append(y_predicte)
    save_object(average_results, "average_results",save_path)
    gc.collect()

  if 'single_agent' in algorithm_type:
    single_agent_result = feature_selection('single_agent', single_agent_agents,chunk_X, chunk_Y)
    f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, single_agent_result)
    over_all.append(f1)
    save_object(single_agent_result, "single_agent_mask_"+str(chunk_number),save_path)
    single_agent_results["f1"].append(f1)
    single_agent_results["recall"].append(recall)
    single_agent_results["precision"].append(precision)
    single_agent_results["accuracy"].append(accuracy)
    single_agent_results["y_predicte"].append(y_predicte)
    save_object(single_agent_results, "single_agent_results",save_path)
    gc.collect()

  if 'random_forest' in algorithm_type:
    random_forest_result = feature_selection('random_forest', random_forest_agents,chunk_X, chunk_Y)
    f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, random_forest_result)
    over_all.append(f1)

    save_object(random_forest_result, "random_forest_mask_"+str(chunk_number),save_path)
    random_forest_results["f1"].append(f1)
    random_forest_results["recall"].append(recall)
    random_forest_results["precision"].append(precision)
    random_forest_results["accuracy"].append(accuracy)
    random_forest_results["y_predicte"].append(y_predicte)
    save_object(random_forest_results, "random_forest_results",save_path)
    gc.collect()

  softmax_result=load_object("softmax_mask_"+str(chunk_number),save_path)
  average_result=load_object("average_mask_"+str(chunk_number),save_path)
  single_agent_result=load_object("single_agent_mask_"+str(chunk_number),save_path)
  random_forest_result=load_object("random_forest_mask_"+str(chunk_number),save_path)

  for softmax,average,single,random in zip(softmax_result,average_result,single_agent_result,random_forest_result):
    sum_votes = sum([softmax,average,single,random])
    if sum_votes > (len(algorithm_type) // 2):result.append(1)
    elif sum_votes == (len(algorithm_type) // 2):
      rand = np.random.uniform(low=0,high=1)
      if rand >0.5:result.append(1)
      else:result.append(0)
    else:result.append(0)

  f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, result)
  over_all.append(f1)
  save_object(result, "voting_mask_"+str(chunk_number),save_path)
  voting_results["f1"].append(f1)
  voting_results["recall"].append(recall)
  voting_results["precision"].append(precision)
  voting_results["accuracy"].append(accuracy)
  voting_results["y_predicte"].append(y_predicte)
  save_object(voting_results, "voting_results",save_path)
  gc.collect()

  re_all = [softmax_result,average_result,single_agent_result,random_forest_result,result]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
def random_forest_feature_selection(X, y):
    """
    return best feature from X using random forest
    """
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 4))
    sel.fit(X, y)
    return sel.get_support()

In [ ]:
def main(f_name,train_size=0.80 , result_save_path="",ChunkNumber=0,feature_selection_type='',algorithm_type=['softmax','average','single_agent','random_forest']):

  datasets = {}
  d = prepare_data(f_name)
  d = d.sample(frac=1, random_state=42)
  buffer = d.sample(n=5000)
  d.reset_index(inplace=True)
  d.replace([np.inf], 0, inplace=True)
  datasets[f_name.split('/')[-1]] = d
  results = {}

  for key in tqdm(datasets.keys()):
      drift_location = {}
      results[key] = {'model_result': []}
      data = datasets[key].values
      X, Y = data[:, 0:-1], data[:, -1].astype('int')
      chunks_features = np.array_split(X, 10)
      chunks_labels = np.array_split(Y, 10)
      print("===================== dataset : {} ======================".format(key))
      for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_features, chunks_labels)):
          try:
            chunk_X, chunk_Y = SMOTE().fit_resample(chunk_X, chunk_Y)
          except:
            if chunk_Y.sum() in [0, 1]:
              new_samples, new_labels = generate_new_samples(buffer, chunk_Y)
              chunk_X = np.concatenate((chunk_X, new_samples))
              chunk_Y = np.concatenate((chunk_Y, new_labels))
          gc.collect()
          X_train, X_test, y_train, y_test = train_test_split(chunk_X, chunk_Y, random_state=42, train_size=train_size)
          # if feature_selection_type == "feature_evolving":
          #   unselected_feautres = feature_evolving(evolving_matrix=evolving_matrix)
            # chunk_X = np.delete(chunk_X, unselected_feautres, 1)

          dynamic_feature_selection(chunk_X=X_train, chunk_Y=y_train,X_test=X_test,y_test=y_test,save_path=result_save_path,chunk_number=CN,algorithm_type=algorithm_type)
          RF_selected = random_forest_feature_selection(X_train, y_train)
          f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, RF_selected)
          print("CHUNK NUMBER ",CN,"DONE")

In [ ]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))
filenames

['/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/kddcup99_csv.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/ISCX2012.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CSE-CIC2018.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CICIDS2017.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7recurrentDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/blip.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/incrementalDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7gradualDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7suddenDrift.csv']

In [ ]:
def results_dic(path,chunk_number=0):
  if chunk_number == 0 :
    softmax_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    average_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    single_agent_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    random_forest_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    voting_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
  else:
    softmax_results = load_object('softmax_results',path)
    average_results = load_object('average_results',path)
    single_agent_results = load_object('single_agent_results',path)
    random_forest_results = load_object('random_forest_results',path)
    voting_results = load_object('voting_results',path)
  return softmax_results,average_results,single_agent_results,random_forest_results,voting_results

In [ ]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [ ]:
# # data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']
# # for f_name,d_name in zip(filenames,data_name):
# #   path = os.path.join(feature_selection_results_evolving,d_name)
# #   os.makedirs(path, exist_ok=True)
# #   main(f_name, result_save_path=path,feature_selection_type="feature_evolving")

In [ ]:
# DATA_NUMBER=0
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=1
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=2
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=3
# ChunkNumber=6
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=4
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=5
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=6
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber,)

In [ ]:
# DATA_NUMBER=7
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=8
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, result_save_path=path,feature_selection_type="feature_evolving",ChunkNumber=ChunkNumber)